In [ ]:
import numpy      as np
import seaborn    as sns
import ML_library as MLL
import torch
import json

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sns.set_theme()

# Some specific validation examples

In [ ]:
input_folder                = 'models'
target_folder               = f'{input_folder}/Fv-OOD'
model_name                  = f'{target_folder}/model.pt'
dataset_parameters_name_std = f'{target_folder}/standardized_parameters.json'

dropout = 0.4

# Load the data from the JSON file
with open(dataset_parameters_name_std, 'r') as json_file:
    numpy_dict = json.load(json_file)

# Convert torch tensors to numpy arrays
dataset_parameters = {}
for key, value in numpy_dict.items():
    try:
        dataset_parameters[key] = torch.tensor(value)
    except:
        dataset_parameters[key] = value

# Load dataset
reference_dataset = torch.load(f'{target_folder}/standardized_dataset.pt')

In [ ]:
# Load Graph Neural Network model (making room for temperature as node attribute)
model = MLL.GCNN(features_channels=4+1, pdropout=dropout)

# Moving model to device
model = model.to(device)

# Load Graph Neural Network model
model.load_state_dict(torch.load(model_name, map_location=torch.device(device)))
model.eval()

In [ ]:
# We read the POSCAR and compare it to the reference database based on the GNN model
target_POSCAR = 'models/Fv-OOD'

# We define the temperature we want compare at
Ti = 500
Tf = 1300
dT = 50

temperatures = np.arange(Ti, Tf+dT, dT)

# Create dataset for predictions
target_dataset, labels = MLL.create_predictions_dataset(target_POSCAR, path_to_material=True, path_to_polymorph=True)

# Standardize properties
target_dataset = MLL.standarize_dataset(target_dataset, dataset_parameters, transformation=dataset_parameters['transformation'])

# Include temperatures
target_dataset = MLL.include_temperatures(target_dataset, temperatures, dataset_parameters)

In [ ]:
# Iterate the same materials over different temperatures
for target_material in target_dataset:
    target_material = target_material.to(device)
    
    # Compute distance from graph to dataset
    distance = MLL.estimate_out_of_distribution(reference_dataset, target_material, model)
    print(distance)

In [ ]:
# POSCAR from dataset at temperatures inside and over dataset

In [ ]:
# POSCAR from dataset with Te -> Rh substitution at temperatures inside and over dataset